In [1]:
import pandas as pd
import numpy as np
import os, copy
import math, time

# Input

1 Read from RSF files of expert clustering and distance graph from excel file

2 Creating list of all clusters

In [2]:
def funcSymmetric(listCall_matrix):
    listTemp_call_matrix = [[0 for i in range (len (listCall_matrix))] for i in range (len (listCall_matrix))]

    for i in range(len(listCall_matrix)):
        for j in range(len(listCall_matrix)):
            listTemp_call_matrix [i][j] = listCall_matrix[i][j] + listCall_matrix[j][i]
    
    for i in range(len(listCall_matrix)):
        for j in range(len(listCall_matrix)):
            if listTemp_call_matrix[i][j] > 1:
                listTemp_call_matrix[i][j] = 1
    
    return listTemp_call_matrix

In [3]:
def funcShortest_path_Floyd(listSymm_call_matrix):
    my_inf = math.inf    
    intLen = len (listSymm_call_matrix)
    
    listShortest_distance_matrix =  copy.deepcopy(listSymm_call_matrix)
    
    for i in range (intLen):
        for j in range (intLen):
            if i != j and listShortest_distance_matrix[i][j] == 0:
                # set infinite to any large number which is bigger than the longest way
                listShortest_distance_matrix[i][j] = my_inf  
                

    for k in range (intLen):
        for i in range (intLen):
            for j in range (intLen):
                if listShortest_distance_matrix[i][j] > (listShortest_distance_matrix[i][k] +\
                                                         listShortest_distance_matrix[k][j]):
                    listShortest_distance_matrix[i][j] = (listShortest_distance_matrix[i][k] +\
                                                          listShortest_distance_matrix[k][j])
    for i in range (intLen):
        listShortest_distance_matrix[i][i] = 0 #the distance between a vertex to itself = 0
    
#     for i in range (intLen): #set 0 to array[i][j] that is no path between them 
#         for j in range (intLen):
#              if i != j and listShortest_distance_matrix[i][j] == my_inf:
#                     listShortest_distance_matrix[i][j] = 0 
                 
    return listShortest_distance_matrix

In [4]:
def funcSet_intDiameter_plus (dfDistanceGraph, intDiameter):
    for i in range(len(dfDistanceGraph)):
        for j in range(len(dfDistanceGraph)):
            if dfDistanceGraph.iloc[i][j] == math.inf:
                dfDistanceGraph.iloc[i][j] = intDiameter + 1
    
    return dfDistanceGraph

In [5]:
def funcInput_distance(strPath_folder):
    dfCall_graph = pd.read_csv(strPath_folder + 'CDG.txt', sep="\t", header=None)
    listSymm_call_matrix = funcSymmetric(dfCall_graph.values.tolist())
    time1 = time.time()
    listDistance_graph = funcShortest_path_Floyd(listSymm_call_matrix)
    dfDistanceGraph = pd.DataFrame(listDistance_graph)
    
    #Diameter of the graph
    intDiameter = max(val for val in dfDistanceGraph.values.ravel() if val < math.inf)
    
    #Set intDiameter+1 to math.inf in dfDistanceGraph
    dfDistanceGraph = funcSet_intDiameter_plus (dfDistanceGraph, intDiameter)
    
    
    #the number of artifacts in the cluster
    intNumber = len(dfDistanceGraph)
    time2 = time.time()
    return dfDistanceGraph, intDiameter, intNumber, time2 - time1

In [6]:
# print(funcInput_distance("result_clustering_mozilla/build/"))

In [7]:
def funcInput_file_name_idx(strPath_folder):
    dfFile_name = pd.read_csv(strPath_folder + 'FileNames.txt', sep="\t", header=None)
    listFile_name_idx = []
    for intIdx in range(len(dfFile_name)):
        listFile_name_idx.extend(dfFile_name.loc[intIdx])
    
    return listFile_name_idx

In [8]:
def funcInput_RSF(strFile_name):

    dfRSF = pd.read_table(strFile_name, sep=' ', header=None)
    
    return dfRSF

In [9]:
def funcInitializing_RSF(dfRSF):
    intNumber = len(dfRSF)
    
    #the names of artifacts in the cluster
    listFileNames = dfRSF[2].tolist()

    #Creating list of all clusters
    #list of Clusters = [[name, list of members]]
    listClusters = []
    for i in range (intNumber):
        if len(listClusters) == 0:
             listClusters.append([dfRSF.iloc[i][1],[dfRSF.iloc[i][2]]])
        else:
            found = False
            for j in range(len(listClusters)):
                if dfRSF.iloc[i][1] in listClusters [j]:
                    listClusters[j][1].append(dfRSF.iloc[i][2])
                    found = True
                    break
            if found == False:
                listClusters.append([dfRSF.iloc[i][1],[dfRSF.iloc[i][2]]])
    
    intK = len(listClusters)
  
    return intK, listClusters, listFileNames
    

In [10]:
def funcDistance_cluster(dfDistanceGraph, listClusters, intDiameter):
    #Computing the (normalized) distances between clusters
    listDistance_clusters = []
    listDistance_clusters_normalized = []
    
    intK = len(listClusters)
    
    for k in range(intK):
        listDistance_clusters.append([])
        listDistance_clusters_normalized.append([])

        for kp in range(intK):
           
            intSum_Distance = sum(dfDistanceGraph.loc[listClusters[k][1][i]][listClusters[kp][1][j]]\
                      for i in range(len(listClusters[k][1])) for j in range(len(listClusters[kp][1])))


            if k != kp:
                listDistance_clusters[-1].append(intSum_Distance)
                listDistance_clusters_normalized[-1].append(intSum_Distance / (intDiameter * len(listClusters[k][1])\
                                                              * len(listClusters[kp][1])))
            else:
                listDistance_clusters[-1].append(intSum_Distance / 2)
                if(len(listClusters[k][1]) == 1):
                    listDistance_clusters_normalized[-1].append(0)
                else:
                    listDistance_clusters_normalized[-1].append(intSum_Distance / \
                                           (intDiameter * len(listClusters[k][1]) * (len(listClusters[k][1])-1)))
    
    return listDistance_clusters, listDistance_clusters_normalized

In [11]:
def funcDistance_cluster2(dfDistanceGraph, listClusters, intDiameter):
    #Computing the (normalized) distances between clusters
    listDistance_clusters = []
#     listDistance_clusters_normalized = []
    
    intK = len(listClusters)
    
    for k in range(intK):
        listDistance_clusters.append([])
#         listDistance_clusters_normalized.append([])

        for kp in range(intK):
           
            intSum_Distance = sum(dfDistanceGraph.loc[listClusters[k][1][i]][listClusters[kp][1][j]]\
                      for i in range(len(listClusters[k][1])) for j in range(len(listClusters[kp][1])))


            if k != kp:
                listDistance_clusters[-1].append(intSum_Distance)
#                 listDistance_clusters_normalized[-1].append(intSum_Distance / (intDiameter * len(listClusters[k][1])\
#                                                               * len(listClusters[kp][1])))
            else:
                listDistance_clusters[-1].append(intSum_Distance / 2)
#                 if(len(listClusters[k][1]) == 1):
#                     listDistance_clusters_normalized[-1].append(0)
#                 else:
#                     listDistance_clusters_normalized[-1].append(intSum_Distance / \
#                                            (intDiameter * len(listClusters[k][1]) * (len(listClusters[k][1])-1)))
    
    return listDistance_clusters#, listDistance_clusters_normalized

In [12]:
def funcInitializing_distance_mask(intK):
    
    #mask for internal and external distance            
    boolMask_distance_internal = np.eye(intK, dtype = bool)
    boolMask_distance_external = np.ones((intK, intK), dtype = bool)
    boolMask_distance_external[np.diag_indices(intK)] = False
    
    return boolMask_distance_internal, boolMask_distance_external
    

In [13]:
def funcRead_param(strPath_input):
    file = strPath_input + 'result.txt'
    dfRead_param = pd.read_csv(file, sep="\t")
    return dfRead_param

In [14]:
def funcPrint_everything(dfDistanceGraph, dfRSF, intNumber, intK, intDiameter, listClusters, listFileNames,\
                         listDistance_clusters, listDistance_clusters_normalized, boolMask_distance_internal,\
                         boolMask_distance_external):
    
    print("dfDistanceGraph")
    print(dfDistanceGraph)

    print("\n\n"+"intDiameter")
    print(intDiameter)    

    print("\n\ndfRSF")
    print(dfRSF)

    print("\n\n"+"listFileNames")
    print(listFileNames)

    print("\n\nlistClusters")
    for i in range(len(listClusters)):
        print((listClusters[i]))

    print("\n\nlistDistance_clusters")
    for i in range(len(listDistance_clusters)):
        for j in range (len(listDistance_clusters[i])):
            print(str(round(listDistance_clusters[i][j] , 2))+"\t" , end="")
        print()


    print("\n\nlistDistance_clusters_normalized")
    for i in range(len(listDistance_clusters_normalized)):
        for j in range (len(listDistance_clusters_normalized[i])):
            print(str(round(listDistance_clusters_normalized[i][j] , 2))+"\t" , end="")
        print()

    print("\n\nboolMask_distance_internal")
    print(boolMask_distance_internal)

    print("\n\nboolMask_distance_external")
    print(boolMask_distance_external)   

    print("\n\n"+"intK")
    print(intK)

    print("\n\n"+"intNumber")
    print(intNumber)

# Criteria

# Criterion 1 : Silhouette Coefficient

a and b is sum of distances
$$S(i) = \frac{b(i) - a(i)}{\max \{a(i) , b(i)\}}$$ 

$$ S = avg \ S(i) \ \ \ \  \forall i $$



In [15]:
def funcCriterion1(intNumber, intK, listClusters, dfDistanceGraph):
    listCriterion1_a = []
    listCriterion1_b = []
    listCriterion1_s = []
    
    #Calculation of listCriterion1_a
    for i in range(intK):
        for j in range(len(listClusters[i][1])):
            if(len(listClusters[i][1])-1 == 0):
                listCriterion1_a.append(0)
            else:
                listCriterion1_a.append(sum(dfDistanceGraph.loc[listClusters[i][1][j]][listClusters[i][1][k]] \
                                        for k in range(len(listClusters[i][1])))/(len(listClusters[i][1])-1))

    
    #Calculation of listCriterion1_b
    listCriterion1_b = []
    temp = []
    for i in range(intK):
        for j in range(len(listClusters[i][1])):
            temp = []
            for k in range(intK):
                if i != k:
                    temp.append(sum(dfDistanceGraph.loc[listClusters[i][1][j]][listClusters[k][1][m]] \
                                    for m in range(len(listClusters[k][1])))/len(listClusters[k][1]))

            listCriterion1_b.append(temp)
    
    #Calculation of Silouette
    listCriterion1_s = []
    for i in range(intNumber):
        listCriterion1_s.append((min(listCriterion1_b[i])-listCriterion1_a[i]) / \
                                max(listCriterion1_a[i],min(listCriterion1_b[i])))
    
    #print("listCriterion1_s")        
    #print(listCriterion1_s)

    S = sum(listCriterion1_s) / intNumber
    #print("\n\nS")
    return S

In [16]:
# print(funcCriterion1(intNumber, intK, listClusters, dfDistanceGraph))

# Criterion 1s : Silhouette Coefficient with similarity

a and b is sum of similarities

$$S(i) = \frac{a(i) - b(i)}{\max \{a(i) , b(i)\}}$$ 

$$ S = avg \ S(i) \ \ \ \  \forall i $$

In [17]:
def createSimilarityMatrix(dfDistanceGraph):
    tempMatrix = [[0 for i in range (len (dfDistanceGraph))] for i in range (len (dfDistanceGraph))]
    v = len (tempMatrix)
    
    maximum = -1    
    for i in range(0,v):
        max_temp = max(dfDistanceGraph.iloc[i])
        if maximum < max_temp:
            maximum = max_temp

    if maximum == 0:
        print("all classes are in its own cluster!")
        exit()
    
    for i in range (0, v):
        for j in range (0, v):
            if dfDistanceGraph.iloc[i][j] == 0 and i !=j:
                tempMatrix[i][j] = maximum
            else:
                tempMatrix[i][j] = dfDistanceGraph.iloc[i][j]
        
    for i in range (0,v):
        for j in range (0, v):
            tempMatrix[i][j] = 1 - tempMatrix[i][j]/maximum
    return tempMatrix

In [18]:
def funcCriterion1s (intNumber, intK, listClusters, dfDistanceGraph, listFileNames):

    listSimilarityMatrix = []
    listSimilarityMatrix = createSimilarityMatrix(dfDistanceGraph)
    
    listCriterion1_a2 = []
    listCriterion1_b2 = []
    listCriterion1_s2 = []
    
    #Calculation of listCriterion1_a2
    for i in range(intK):
        for j in range(len(listClusters[i][1])):
            if(len(listClusters[i][1])-1 == 0):
                listCriterion1_a2.append(1)
            else:
                listCriterion1_a2.append((sum(listSimilarityMatrix[listFileNames.index(listClusters[i][1][j])]\
                                          [listFileNames.index(listClusters[i][1][k])] \
                                          for k in range(len(listClusters[i][1])))-1)/(len(listClusters[i][1])-1))
    
    
    #Calculation of listCriterion1_b2
    temp = []

    for i in range(intK):
        for j in range(len(listClusters[i][1])):
            temp = []
            for k in range(intK):
                if i != k:
                    temp.append(sum(listSimilarityMatrix[listFileNames.index(listClusters[i][1][j])]\
                                    [listFileNames.index(listClusters[k][1][m])] \
                                    for m in range(len(listClusters[k][1])))/len(listClusters[k][1]))

            listCriterion1_b2.append(temp)

        
    for i in range(intNumber):
        listCriterion1_s2.append((listCriterion1_a2[i]-max(listCriterion1_b2[i])) / \
                                 max(listCriterion1_a2[i],max(listCriterion1_b2[i])))

    #print("listCriterion1_s2")    
    #print(listCriterion1_s2)
        
    S = sum(listCriterion1_s2) / intNumber
    #print("\n\nS")
    #print(S)
    
    return S


In [19]:
# print(funcCriterion1s (intNumber, intK, listClusters, dfDistanceGraph, listFileNames))

## Criterion 2

A: the total amount of distances between all pairs of vertices in different clusters

B: the total amount of distances between all pairs of vertices in the same cluster

$$C = \frac{A - B} {\max \{A , B\}}$$

In [20]:
def funcCriterion2(listDistance_clusters, boolMask_distance_internal, boolMask_distance_external):

    intCriterion2_A = 0
    intCriterion2_B = 0

    intCriterion2_A = sum(np.array(listDistance_clusters)[boolMask_distance_external])/2
    intCriterion2_B = sum(np.array(listDistance_clusters)[boolMask_distance_internal])

    intCriterion2_C = (intCriterion2_A - intCriterion2_B) / max(intCriterion2_A , intCriterion2_B)
    
    return intCriterion2_C

## Criterion 2n

A: the total amount of normalized distances between all pairs of vertices in different clusters

B: the total amount of normalized distances between all pairs of vertices in the same cluster

$$C = \frac{A - B} {\max \{A , B\}}$$

In [21]:
def funcCriterion2n(listDistance_clusters_normalized, boolMask_distance_internal, boolMask_distance_external, intK):

    intCriterion2_A = 0
    intCriterion2_B = 0

    intCriterion2_A = sum(np.array(listDistance_clusters_normalized)[boolMask_distance_external])/(intK * (intK - 1))
    intCriterion2_B = sum(np.array(listDistance_clusters_normalized)[boolMask_distance_internal]) / intK

    intCriterion2_C = (intCriterion2_A - intCriterion2_B) / max(intCriterion2_A , intCriterion2_B)
    
    return intCriterion2_C

In [22]:
# print(funcCriterion2(listDistance_clusters, boolMask_distance_internal, boolMask_distance_external))
# print(funcCriterion2(listDistance_clusters_normalized, boolMask_distance_internal, boolMask_distance_external))

## Criterion 3

intCriterion3_internal [i]: the total amount of distances between all pairs of vertices in cluster i 

intCriterion3_external [i,j]: the total amount of distances between all pairs of vertices in cluster [i] and cluster [j]

$$C = \frac{\sum_{i = 1}^{k}\sum_{j = i}^{k} external\ [i,j]}{\frac{k (k - 1)}{2}} - \frac{\sum_{i = 1}^{k} internal \ [i]}{k} $$ 



In [23]:
def funcCriterion3(listDistance_clusters, boolMask_distance_internal, boolMask_distance_external, intK):
    intCriterion3_external = 0
    intCriterion3_internal = 0
    intCriterion3_external = sum(np.array(listDistance_clusters)[boolMask_distance_external])/2
    intCriterion3_internal = sum(np.array(listDistance_clusters)[boolMask_distance_internal])

    intCriterion3_C = intCriterion3_external / (intK * (intK - 1)/2) - intCriterion3_internal / intK
    return intCriterion3_C

In [24]:
# print(funcCriterion3(listDistance_clusters, boolMask_distance_internal, boolMask_distance_external, intK))

## Criterion 4

internal_i: the average amount of distances between all pairs of cluster C_i. The average is computed by dividing by the diameter of the graph multiplied by the number of connections in the very cluster.

external_i: the average amount of distances between all pairs of cluster C_i and the other clusters. The average is computed as above.

$$Criterion 4 = avg(external_i - internal_i) \ \ \ \ \forall C_i$$

In [25]:
def funcCriterion4(listDistance_clusters_normalized, boolMask_distance_internal, \
                             boolMask_distance_external, intK):
    listCriterion4_external = []
    listCriterion4_internal = []
    listDistance_external = []
  
    listCriterion4_internal = np.array(listDistance_clusters_normalized)[boolMask_distance_internal]
    #print(listCriterion4_internal)
    
    #print(np.array(listDistance_clusters_normalized))
    listDistance_external = np.array(listDistance_clusters_normalized)[boolMask_distance_external].\
                            reshape(intK,intK-1)
    #print(listDistance_external)
    for i in range(intK):
        listCriterion4_external.append(sum(listDistance_external[i])/(intK-1))
    #print(listCriterion4_external)

    
    listCriterion4_T = [] #for computing the mean of A_i and B_i
    for k in range (intK):
        listCriterion4_T.append(listCriterion4_external[k] - listCriterion4_internal[k])
    intCriterion4 = np.mean(listCriterion4_T)
    return intCriterion4


In [26]:
# print(funcCriterion4(intDiameter, listClusters, dfDistanceGraph))

## Criterion 5

internal: the maximum value of distances between all vertices in the same cluster. The average is computed by dividing by diameter of the graph multiplied by the number of connections in the very cluster.

external: the minimum value of distances between all pairs of clusters. The average is computed as above.

$$Criterion 5 = \frac{external - internal}{max\{external , internal \}} $$

In [27]:
def funcCriterion5(listDistance_clusters_normalized, boolMask_distance_internal, boolMask_distance_external):

    #print(listCriterion5_internal)
    intMax_Internal = max(np.array(listDistance_clusters_normalized)[boolMask_distance_internal])

    #print(listCriterion5_external)
    intMin_external = min(np.array(listDistance_clusters_normalized)[boolMask_distance_external])

    intCriterion5 = (intMin_external - intMax_Internal)/(max(intMax_Internal, intMin_external))
    
    return intCriterion5

In [28]:
# print(funcCriterion5(listDistance_clusters_normalized, boolMask_distance_internal, boolMask_distance_external))

## Criterion 5u (unbiased)

internal: the maximum value of distances between all vertices in the same cluster. 

external: the minimum value of distances between all pairs of clusters. 

$$Criterion 5u = \frac{external - internal}{max\{external , internal \}} $$

In [29]:
def funcCriterion5u(listDistance_clusters, boolMask_distance_internal, boolMask_distance_external):
    
    #print(intMax_Internal_u)
    intMax_Internal_u = max(np.array(listDistance_clusters)[boolMask_distance_internal])

    #print(intMin_external_u)
    intMin_external_u = min(np.array(listDistance_clusters)[boolMask_distance_external])

    intCriterion5_u = (intMin_external_u - intMax_Internal_u)/(max(intMax_Internal_u, intMin_external_u))   
    return intCriterion5_u

In [30]:
# print(funcCriterion5u(listDistance_clusters, boolMask_distance_internal, boolMask_distance_external))

## Criterion 6

Sum of normalized distance netween any cluster and its neighbour cluster

$$Criterion 6 = \sum{Normalized\ Distance\ (C_i, neigh(C_i))}  \ \ \ \forall C_i  $$

In [31]:
def funcCriterion6(listDistance_clusters_normalized, boolMask_distance_external, intK):
    
    listCriterion6_distance = np.array(listDistance_clusters_normalized)[boolMask_distance_external].\
                                reshape(intK, intK-1)

    intCriterion6 = sum(min(listCriterion6_distance[i]) for i in range(intK))/intK
    
    return intCriterion6

In [32]:
# print(funcCriterion6(listDistance_clusters_normalized, boolMask_distance_external, intK))

## Criterion 7

1 compute the criterion 2 for obtained clustering

2 merge the most similar pair of clusters into one cluster 

3 compute the criterion 2 for obtained clustering, again

4 compute the difference

In [33]:
def funcCriterion7(listDistance_clusters, listDistance_clusters_normalized, boolMask_distance_internal, \
                   boolMask_distance_external, listClusters, dfDistanceGraph, intDiameter):
    
    intK = len(listClusters)
    ###calculating the criterion 2 before merging
    intCriterion_2_Before = funcCriterion2(listDistance_clusters, boolMask_distance_internal, \
                                           boolMask_distance_external)
    
    #for i in range(len(listClusters)):
    #    print(listClusters[i])
        
    #print(intCriterion_2_Before)
    
    listDistance_external = np.array(listDistance_clusters_normalized)[boolMask_distance_external]\
                            .reshape(intK,intK-1)
    (intIdx_row, intIdx_col) = np.unravel_index(listDistance_external.argmin(), listDistance_external.shape)
    
    ###determining the exact index of clusters to be merged
    if intIdx_row >= intIdx_col:
        intIdx_col = intIdx_col + 1
    
    ###merging the clusters
    listClusters_merge = copy.deepcopy(listClusters)
    listClusters_merge[intIdx_row][0] = listClusters_merge[intIdx_row][0]+'m'
    listClusters_merge[intIdx_row][1].extend(listClusters_merge[intIdx_col][1])
    del listClusters_merge[intIdx_col]
    
    #for i in range(len(listClusters_merge)):
    #    print(listClusters_merge[i])
    
    ###calculating the criterion 2 after merging
    listDistance_clusters_merge, listDistance_clusters_merge_normalized = \
                funcDistance_cluster(dfDistanceGraph, listClusters_merge, intDiameter)
    
    
    boolMask_distance_internal, boolMask_distance_external = funcInitializing_distance_mask(intK-1)

    intCriterion_2_after = funcCriterion2(listDistance_clusters_merge, boolMask_distance_internal, \
                                           boolMask_distance_external)
    

    #print(intCriterion_2_after)    
    return intCriterion_2_after - intCriterion_2_Before

## Criterion 7n

1 compute the criterion 2n for obtained clustering

2 merge the most similar pair of clusters into one cluster 

3 compute the criterion 2n for obtained clustering, again

4 compute the difference

In [34]:
def funcCriterion7n(listDistance_clusters, listDistance_clusters_normalized, boolMask_distance_internal, \
                   boolMask_distance_external, listClusters, dfDistanceGraph, intDiameter):
    
    intK = len(listClusters)
    ###calculating the criterion 2 before merging
    intCriterion_2_Before = funcCriterion2n(listDistance_clusters_normalized, boolMask_distance_internal, \
                                           boolMask_distance_external, intK)
    
    #for i in range(len(listClusters)):
    #    print(listClusters[i])
        
    #print(intCriterion_2_Before)
    
    listDistance_external = np.array(listDistance_clusters_normalized)[boolMask_distance_external]\
                            .reshape(intK,intK-1)
    (intIdx_row, intIdx_col) = np.unravel_index(listDistance_external.argmin(), listDistance_external.shape)
    
    ###determining the exact index of clusters to be merged
    if intIdx_row >= intIdx_col:
        intIdx_col = intIdx_col + 1
    
    ###merging the clusters
    listClusters_merge = copy.deepcopy(listClusters)
    listClusters_merge[intIdx_row][0] = listClusters_merge[intIdx_row][0]+'m'
    listClusters_merge[intIdx_row][1].extend(listClusters_merge[intIdx_col][1])
    del listClusters_merge[intIdx_col]
    
    #for i in range(len(listClusters_merge)):
    #    print(listClusters_merge[i])
    
    ###calculating the criterion 2 after merging
    listDistance_clusters_merge, listDistance_clusters_merge_normalized = \
                funcDistance_cluster(dfDistanceGraph, listClusters_merge, intDiameter)
    
    boolMask_distance_internal, boolMask_distance_external = funcInitializing_distance_mask(intK-1)

    intCriterion_2_after = funcCriterion2n(listDistance_clusters_merge_normalized, boolMask_distance_internal, \
                                           boolMask_distance_external, intK)
    

    #print(intCriterion_2_after)    
    return intCriterion_2_after - intCriterion_2_Before

In [35]:
# print(ffuncCriterion7(listDistance_clusters, boolMask_distance_internal, boolMask_distance_external, listClusters,\
#                   dfDistanceGraph, intDiameter))

# Start

In [36]:
def funcStart():
    listPath_best_NO_cluster = []
    listPath_best_NO_cluster.append(["result_clustering_mozilla/build/", 2])
    listPath_best_NO_cluster.append(["result_clustering_mozilla/accessible/", 9])
    listPath_best_NO_cluster.append(["result_clustering_mozilla/browser/", 8])
    listPath_best_NO_cluster.append(["result_clustering_mozilla/db/", 4])
    listPath_best_NO_cluster.append(["result_clustering_mozilla/dom/", 5])
    listPath_best_NO_cluster.append(["result_clustering_mozilla/extensions/", 13])
    listPath_best_NO_cluster.append(["result_clustering_mozilla/gfx/", 8])
    listPath_best_NO_cluster.append(["result_clustering_mozilla/intl/", 7])
    listPath_best_NO_cluster.append(["result_clustering_mozilla/ipc/", 4])
    listPath_best_NO_cluster.append(["result_clustering_mozilla/content/", 13])

    
    listHeader_excel =  ["K", \
                     "RunNO", \
                     "MoJo", \
                     "MoJo_fm", \
                     "Floyd_time", \
                     "Distance_Clusters_time", \
                     "Silouette_distance", \
                     "Sil_time", \
                     "Criterion_2", \
                     "C_2_time", \
                     "Criterion_3", \
                     "C_3_time", \
                     "Criterion_4", \
                     "C_4_time", \
                     "Criterion_5", \
                     "C_5_time", \
                     "Criterion_6", \
                     "C_6_time", \
                     "Criterion_7", \
                     "C_7time"]
    
    for intFolder_idx in range (len(listPath_best_NO_cluster)):

        dfDistanceGraph, intDiameter, intNumber, intTime_Floyd = funcInput_distance(listPath_best_NO_cluster [intFolder_idx][0])
        listFile_name_idx = funcInput_file_name_idx(listPath_best_NO_cluster [intFolder_idx][0])
        
        dfSelected = funcRead_param(listPath_best_NO_cluster [intFolder_idx][0])
        #print(dfSelected)

        strPrefix_file_name = listPath_best_NO_cluster[intFolder_idx][0]
        
        ###list for storing every creterion
        listCriteria = []
        
        listFile_path_k_runno = []
        
        listFile_path_k_runno.append([strPrefix_file_name + "MoJoExpert.txt", \
                                      listPath_best_NO_cluster[intFolder_idx][1], 0, 0, 100])
        
        for intIdx in dfSelected.T:
            listFile_path_k_runno.append([strPrefix_file_name + "MoJoAlgorithm" + str(int(dfSelected.loc[intIdx]['K'])) + "_"\
                          + str(int(dfSelected.loc[intIdx]['RunNO'])) + ".txt",\
                                          int(dfSelected.loc[intIdx]['K']), \
                                          int(dfSelected.loc[intIdx]['RunNO']),\
                                          int(dfSelected.loc[intIdx]['MoJo']),\
                                          int(dfSelected.loc[intIdx]['MoJo_fm'])])

        

#         print("listFile_name_idx")
#         print(listFile_name_idx)
#         print("dfDistanceGraph")
#         print(dfDistanceGraph)

        dfDistanceGraph.index = listFile_name_idx
        dfDistanceGraph.rename(columns = {i: listFile_name_idx[i] for i in range(intNumber)}, inplace=True)

        
         
#         print("\n\n"+"intDiameter")
#         print(intDiameter)    

#         print("\n\n"+"intNumber")
#         print(intNumber)

        
        
        
        
        
        for intIdx in range(len(listFile_path_k_runno)):
            print("processing\t" + str(intIdx) + ":  " + listFile_path_k_runno[intIdx][0])
            dfRSF = funcInput_RSF(listFile_path_k_runno[intIdx][0])

#             print("\n\ndfRSF")
#             print(dfRSF)
            
            intK, listClusters, listFile_names = funcInitializing_RSF(dfRSF)
            
#             print("\n\n"+"listFile_names")
#             print(listFile_names)

#             print("\n\nlistClusters")
#             for i in range(len(listClusters)):
#                 print((listClusters[i]))

            
#             print("\n\n"+"intK")
#             print(intK)

            
#             listDistance_clusters = funcDistance_cluster2(dfDistanceGraph, listClusters, intDiameter)
            
            
            time1 = time.time()
            listDistance_clusters, listDistance_clusters_normalized = \
                        funcDistance_cluster(dfDistanceGraph, listClusters, intDiameter)
            time2 = time.time()
            intTime_Distance_clusters = time2 - time1

#             print("\n\nlistDistance_clusters")
#             for i in range(len(listDistance_clusters)):
#                 for j in range (len(listDistance_clusters[i])):
#                     print(str(round(listDistance_clusters[i][j] , 2))+"\t" , end="")
#                 print()


#             print("\n\nlistDistance_clusters_normalized")
#             for i in range(len(listDistance_clusters_normalized)):
#                 for j in range (len(listDistance_clusters_normalized[i])):
#                     print(str(round(listDistance_clusters_normalized[i][j] , 2))+"\t" , end="")
#                 print()

            
            
            boolMask_distance_internal, boolMask_distance_external = funcInitializing_distance_mask(intK)
            
#             print("\n\nboolMask_distance_internal")
#             print(boolMask_distance_internal)

#             print("\n\nboolMask_distance_external")
#             print(boolMask_distance_external)             
            
            
            
            
            listCriteria.append([])
 
            listCriteria[-1].append(int(listFile_path_k_runno[intIdx][1])) #k
            listCriteria[-1].append(int(listFile_path_k_runno[intIdx][2])) #Run_no
            listCriteria[-1].append(int(listFile_path_k_runno[intIdx][3])) #Mojo
            listCriteria[-1].append(int(listFile_path_k_runno[intIdx][4])) #Mojo_fm
            listCriteria[-1].append(intTime_Floyd) #intTime_Floyd
            listCriteria[-1].append(intTime_Distance_clusters) #intTime_Distance_clusters
            
            
            time1 = time.time()
            C_1 = funcCriterion1(intNumber, intK, listClusters, dfDistanceGraph)
            time2 = time.time()
            intTime_C_1 = time2 - time1
            listCriteria[-1].append(C_1)
            listCriteria[-1].append(intTime_C_1)
            
            
            time1 = time.time()
            C_2 = funcCriterion2(listDistance_clusters, boolMask_distance_internal, boolMask_distance_external)
            time2 = time.time()
            intTime_C_2 = time2 - time1
            listCriteria[-1].append(C_2)
            listCriteria[-1].append(intTime_C_2)        
            
            
            time1 = time.time()
            C_3 = funcCriterion3(listDistance_clusters, boolMask_distance_internal, boolMask_distance_external, intK)
            time2 = time.time()
            intTime_C_3 = time2 - time1
            listCriteria[-1].append(C_3)
            listCriteria[-1].append(intTime_C_3)        
        
        
            time1 = time.time()
            C_4 = funcCriterion4(listDistance_clusters_normalized, boolMask_distance_internal, boolMask_distance_external, intK)
            time2 = time.time()
            intTime_C_4 = time2 - time1
            listCriteria[-1].append(C_4)
            listCriteria[-1].append(intTime_C_4)           
        
        
            time1 = time.time()
            C_5 = funcCriterion5(listDistance_clusters_normalized, boolMask_distance_internal, boolMask_distance_external)
            time2 = time.time()
            intTime_C_5 = time2 - time1
            listCriteria[-1].append(C_5)
            listCriteria[-1].append(intTime_C_5)     
        
        
            time1 = time.time()
            C_6 = funcCriterion6(listDistance_clusters_normalized, boolMask_distance_external, intK)
            time2 = time.time()
            intTime_C_6 = time2 - time1
            listCriteria[-1].append(C_6)
            listCriteria[-1].append(intTime_C_6)     
                
            
            time1 = time.time()   
            C_7 = funcCriterion7(listDistance_clusters, listDistance_clusters_normalized, \
                                                   boolMask_distance_internal, boolMask_distance_external, \
                                                   listClusters, dfDistanceGraph, intDiameter)
            time2 = time.time()
            intTime_C_7 = time2 - time1
            listCriteria[-1].append(C_7)
            listCriteria[-1].append(intTime_C_7)  
            
            #if intIdx == 1:
            #    break

        #for i in range(len(listCriteria)):
        #    print(listCriteria[i])

        ###storing in a data frame
        funcStore_df (listCriteria, listHeader_excel, strPrefix_file_name)
        
        

## Store in Excel

In [37]:
def funcStore_df (listCriteria, listHeader_excel, strPrefix_file_name):
    writer = pd.ExcelWriter(strPrefix_file_name + 'Criteria.xlsx')
    dfOutput = pd.DataFrame(listCriteria, columns=listHeader_excel)
    print(dfOutput)
    dfOutput.to_excel(writer, 'Criteria')
    writer.save()

In [38]:
funcStart()

processing	0:  result_clustering_mozilla/build/MoJoExpert.txt
processing	1:  result_clustering_mozilla/build/MoJoAlgorithm2_1.txt
processing	2:  result_clustering_mozilla/build/MoJoAlgorithm3_1.txt
processing	3:  result_clustering_mozilla/build/MoJoAlgorithm4_1.txt
processing	4:  result_clustering_mozilla/build/MoJoAlgorithm5_1.txt
processing	5:  result_clustering_mozilla/build/MoJoAlgorithm6_1.txt
processing	6:  result_clustering_mozilla/build/MoJoAlgorithm7_1.txt
   K  RunNO  MoJo  MoJo_fm  Floyd_time  Distance_Clusters_time  \
0  2      0     0      100    0.363267                0.136487   
1  2      1     4       78    0.363267                0.085267   
2  3      1     5       73    0.363267                0.114874   
3  4      1     6       68    0.363267                0.136530   
4  5      1     7       63    0.363267                0.110702   
5  6      1     6       68    0.363267                0.158490   
6  7      1     6       68    0.363267                0.137259   

 

processing	0:  result_clustering_mozilla/browser/MoJoExpert.txt
processing	1:  result_clustering_mozilla/browser/MoJoAlgorithm2_1.txt
processing	2:  result_clustering_mozilla/browser/MoJoAlgorithm3_1.txt
processing	3:  result_clustering_mozilla/browser/MoJoAlgorithm4_1.txt
processing	4:  result_clustering_mozilla/browser/MoJoAlgorithm5_1.txt
processing	5:  result_clustering_mozilla/browser/MoJoAlgorithm6_1.txt
processing	6:  result_clustering_mozilla/browser/MoJoAlgorithm7_1.txt
processing	7:  result_clustering_mozilla/browser/MoJoAlgorithm8_1.txt
processing	8:  result_clustering_mozilla/browser/MoJoAlgorithm9_1.txt
processing	9:  result_clustering_mozilla/browser/MoJoAlgorithm10_1.txt
processing	10:  result_clustering_mozilla/browser/MoJoAlgorithm11_1.txt
processing	11:  result_clustering_mozilla/browser/MoJoAlgorithm12_1.txt
processing	12:  result_clustering_mozilla/browser/MoJoAlgorithm13_1.txt
processing	13:  result_clustering_mozilla/browser/MoJoAlgorithm14_1.txt
processing	14:  r

processing	0:  result_clustering_mozilla/dom/MoJoExpert.txt
processing	1:  result_clustering_mozilla/dom/MoJoAlgorithm2_1.txt
processing	2:  result_clustering_mozilla/dom/MoJoAlgorithm3_1.txt
processing	3:  result_clustering_mozilla/dom/MoJoAlgorithm4_1.txt
processing	4:  result_clustering_mozilla/dom/MoJoAlgorithm5_1.txt
processing	5:  result_clustering_mozilla/dom/MoJoAlgorithm6_1.txt
processing	6:  result_clustering_mozilla/dom/MoJoAlgorithm7_1.txt
processing	7:  result_clustering_mozilla/dom/MoJoAlgorithm8_1.txt
processing	8:  result_clustering_mozilla/dom/MoJoAlgorithm9_1.txt
processing	9:  result_clustering_mozilla/dom/MoJoAlgorithm10_1.txt
processing	10:  result_clustering_mozilla/dom/MoJoAlgorithm11_1.txt
processing	11:  result_clustering_mozilla/dom/MoJoAlgorithm12_1.txt
processing	12:  result_clustering_mozilla/dom/MoJoAlgorithm13_1.txt
processing	13:  result_clustering_mozilla/dom/MoJoAlgorithm14_1.txt
processing	14:  result_clustering_mozilla/dom/MoJoAlgorithm15_1.txt
proce

processing	0:  result_clustering_mozilla/extensions/MoJoExpert.txt
processing	1:  result_clustering_mozilla/extensions/MoJoAlgorithm2_1.txt
processing	2:  result_clustering_mozilla/extensions/MoJoAlgorithm3_1.txt
processing	3:  result_clustering_mozilla/extensions/MoJoAlgorithm4_1.txt
processing	4:  result_clustering_mozilla/extensions/MoJoAlgorithm5_1.txt
processing	5:  result_clustering_mozilla/extensions/MoJoAlgorithm6_1.txt
processing	6:  result_clustering_mozilla/extensions/MoJoAlgorithm7_1.txt
processing	7:  result_clustering_mozilla/extensions/MoJoAlgorithm8_1.txt
processing	8:  result_clustering_mozilla/extensions/MoJoAlgorithm9_1.txt
processing	9:  result_clustering_mozilla/extensions/MoJoAlgorithm10_1.txt
processing	10:  result_clustering_mozilla/extensions/MoJoAlgorithm11_1.txt
processing	11:  result_clustering_mozilla/extensions/MoJoAlgorithm12_1.txt
processing	12:  result_clustering_mozilla/extensions/MoJoAlgorithm13_1.txt
processing	13:  result_clustering_mozilla/extensio

processing	0:  result_clustering_mozilla/gfx/MoJoExpert.txt
processing	1:  result_clustering_mozilla/gfx/MoJoAlgorithm2_1.txt
processing	2:  result_clustering_mozilla/gfx/MoJoAlgorithm3_1.txt
processing	3:  result_clustering_mozilla/gfx/MoJoAlgorithm4_1.txt
processing	4:  result_clustering_mozilla/gfx/MoJoAlgorithm5_1.txt
processing	5:  result_clustering_mozilla/gfx/MoJoAlgorithm6_1.txt
processing	6:  result_clustering_mozilla/gfx/MoJoAlgorithm7_1.txt
processing	7:  result_clustering_mozilla/gfx/MoJoAlgorithm8_1.txt
processing	8:  result_clustering_mozilla/gfx/MoJoAlgorithm9_1.txt
processing	9:  result_clustering_mozilla/gfx/MoJoAlgorithm10_1.txt
processing	10:  result_clustering_mozilla/gfx/MoJoAlgorithm11_1.txt
processing	11:  result_clustering_mozilla/gfx/MoJoAlgorithm12_1.txt
processing	12:  result_clustering_mozilla/gfx/MoJoAlgorithm13_1.txt
processing	13:  result_clustering_mozilla/gfx/MoJoAlgorithm14_1.txt
processing	14:  result_clustering_mozilla/gfx/MoJoAlgorithm15_1.txt
proce

processing	0:  result_clustering_mozilla/intl/MoJoExpert.txt
processing	1:  result_clustering_mozilla/intl/MoJoAlgorithm2_1.txt
processing	2:  result_clustering_mozilla/intl/MoJoAlgorithm3_1.txt
processing	3:  result_clustering_mozilla/intl/MoJoAlgorithm4_1.txt
processing	4:  result_clustering_mozilla/intl/MoJoAlgorithm5_1.txt
processing	5:  result_clustering_mozilla/intl/MoJoAlgorithm6_1.txt
processing	6:  result_clustering_mozilla/intl/MoJoAlgorithm7_1.txt
processing	7:  result_clustering_mozilla/intl/MoJoAlgorithm8_1.txt
processing	8:  result_clustering_mozilla/intl/MoJoAlgorithm9_1.txt
processing	9:  result_clustering_mozilla/intl/MoJoAlgorithm10_1.txt
processing	10:  result_clustering_mozilla/intl/MoJoAlgorithm11_1.txt
processing	11:  result_clustering_mozilla/intl/MoJoAlgorithm12_1.txt
processing	12:  result_clustering_mozilla/intl/MoJoAlgorithm13_1.txt
processing	13:  result_clustering_mozilla/intl/MoJoAlgorithm14_1.txt
processing	14:  result_clustering_mozilla/intl/MoJoAlgorith

processing	0:  result_clustering_mozilla/ipc/MoJoExpert.txt
processing	1:  result_clustering_mozilla/ipc/MoJoAlgorithm2_1.txt
processing	2:  result_clustering_mozilla/ipc/MoJoAlgorithm3_1.txt
processing	3:  result_clustering_mozilla/ipc/MoJoAlgorithm4_1.txt
processing	4:  result_clustering_mozilla/ipc/MoJoAlgorithm5_1.txt
processing	5:  result_clustering_mozilla/ipc/MoJoAlgorithm6_1.txt
processing	6:  result_clustering_mozilla/ipc/MoJoAlgorithm7_1.txt
processing	7:  result_clustering_mozilla/ipc/MoJoAlgorithm8_1.txt
processing	8:  result_clustering_mozilla/ipc/MoJoAlgorithm9_1.txt
processing	9:  result_clustering_mozilla/ipc/MoJoAlgorithm10_1.txt
processing	10:  result_clustering_mozilla/ipc/MoJoAlgorithm11_1.txt
processing	11:  result_clustering_mozilla/ipc/MoJoAlgorithm12_1.txt
processing	12:  result_clustering_mozilla/ipc/MoJoAlgorithm13_1.txt
processing	13:  result_clustering_mozilla/ipc/MoJoAlgorithm14_1.txt
processing	14:  result_clustering_mozilla/ipc/MoJoAlgorithm15_1.txt
proce

processing	0:  result_clustering_mozilla/content/MoJoExpert.txt
processing	1:  result_clustering_mozilla/content/MoJoAlgorithm2_1.txt
processing	2:  result_clustering_mozilla/content/MoJoAlgorithm3_1.txt
processing	3:  result_clustering_mozilla/content/MoJoAlgorithm4_1.txt
processing	4:  result_clustering_mozilla/content/MoJoAlgorithm5_1.txt
processing	5:  result_clustering_mozilla/content/MoJoAlgorithm6_1.txt
processing	6:  result_clustering_mozilla/content/MoJoAlgorithm7_1.txt
processing	7:  result_clustering_mozilla/content/MoJoAlgorithm8_1.txt
processing	8:  result_clustering_mozilla/content/MoJoAlgorithm9_1.txt
processing	9:  result_clustering_mozilla/content/MoJoAlgorithm10_1.txt
processing	10:  result_clustering_mozilla/content/MoJoAlgorithm11_1.txt
processing	11:  result_clustering_mozilla/content/MoJoAlgorithm12_1.txt
processing	12:  result_clustering_mozilla/content/MoJoAlgorithm13_1.txt
processing	13:  result_clustering_mozilla/content/MoJoAlgorithm14_1.txt
processing	14:  r